### ***Importando as bibliotecas***

In [1]:
import torch
import gc
from datasets import load_dataset
from unsloth import FastLanguageModel, is_bfloat16_supported
from trl import SFTTrainer
from transformers import TrainingArguments

Unsloth: Your Flash Attention 2 installation seems to be broken?
A possible explanation is you have a new CUDA version which isn't
yet compatible with FA2? Please file a ticket to Unsloth or FA2.
We shall now use Xformers instead, which does not have any performance hits!
We found this negligible impact by benchmarking on 1x A100.
🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


### ***Inicializando o Foundation Model***

In [2]:
max_seq_length = 256 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

### ***Carregando os modelos***

Carrega uma lista com o nome de todos os modelos que foram treinados na etapa 4.

In [3]:
# Lista com os nomes dos arquivos
modelos = [f'./model/lora_model_parte_{num}' for num in range(1, 10)]
modelos.append('./model/lora_model_final')
print(modelos)

['./model/lora_model_parte_1', './model/lora_model_parte_2', './model/lora_model_parte_3', './model/lora_model_parte_4', './model/lora_model_parte_5', './model/lora_model_parte_6', './model/lora_model_parte_7', './model/lora_model_parte_8', './model/lora_model_parte_9', './model/lora_model_final']


### ***Carregando o dataset de teste***

Carrega o dataset que será utilizado para a avaliação.

In [4]:
# Caminho do arquivo JSON do dataset de teste
caminho_arquivo = './LF-Amazon-1.3M/teste/tst_formatado.json'

# Carregar o dataset diretamente do arquivo JSON
dataset = load_dataset("json", data_files=caminho_arquivo, split="train")

print(dataset[:5])

Generating train split: 0 examples [00:00, ? examples/s]

{'uid': ['B003YI166W', 'B0014DU2QY', 'B00523JL5K', 'B000AOEPMK', '0764599100'], 'title': ['Dreambaby Refrigerator Latch', 'Pacon Classroom Select Pacon Sentence Strip, 3 X 24 in, 1-1/2 in Ruling, 3/4 in Dotted Midline, 1 in Descender Space, Assorted Colors, Pack of 100', 'HTC EVO 3D Accessory - Blossoming Purple Violet Flower Protective Hard Case Cover Design for Sprint 4G', 'The Martha Stewart Holiday Collection - Homemade Holidays', 'Card Games For Dummies'], 'content': ['The Dream Baby Refrigerator Latch helps prevent children from opening fridges or freezers.', 'Pacon Classroom Select Assorted Super Bright Sentence Strip measuring 3 in x 24 in contains 1-1/2 in blue ruling, 3/4 in dotted midline and 1 in descender space. Strip is sold as 100 per pack.', 'Protect your cell phone with our premium Crystal Image case. This accessory provides protection for your cell phone from unnecessary scratch, dent or chips. It is made to fit perfectly and give the phone maximum protection. The har

### ***Avaliando***

**Argumentos de avaliação**

A única diferença é a adição dos seguintes parâmetros:
- *do_train*: Desabilita o treinamento.
- *do_eval*: Realiza a avaliação.

In [5]:
# Parâmetros de avaliação
trainingArguments = TrainingArguments(
    per_device_train_batch_size=128,
    gradient_accumulation_steps=2,
    warmup_steps=5,
    num_train_epochs=1,
    max_steps=60,
    learning_rate=3e-4,
    fp16=not is_bfloat16_supported(),
    bf16=is_bfloat16_supported(),
    logging_steps=20,
    optim="adamw_8bit",
    weight_decay=0.01,
    lr_scheduler_type="linear",
    seed=3407,
    output_dir="outputs",
    do_train=False,  # Desabilita o treinamento
    do_eval=True
)

**Função de avaliação**

Realiza os seguintes passos:

**1.** Limpeza de memória
- Realiza a limpeza da memória antes de iniciar o modelo.

**2.** Inicialização do modelo.
- Inicia o modelo e o tokenizer passado por parâmetro.

**3.** Criação do *trainer*
- Cria o trainer passando apenas o dataset de avaliação.

**4.** Avaliação do modelo
- Inicia a avaliazação do modelo com o dataset que foi passado.

In [6]:
def avaliar_e_limpar_memoria(modelo):
    print(f"Avaliando o modelo: {modelo}")
    
    torch.cuda.empty_cache()  # Limpar a memória da GPU
    gc.collect()  # Forçar a coleta de lixo
    
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = modelo,
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    
    # Configurar o treinador para cada parte do dataset
    trainer = SFTTrainer(
        model=model,
        tokenizer=tokenizer,
        eval_dataset=dataset,
        dataset_text_field="text",
        max_seq_length=max_seq_length,
        dataset_num_proc=8,  # Multiprocessamento
        packing=False,  # Se necessário para sequências curtas
        args=trainingArguments
    )
    
    # Avaliar o modelo
    eval_results = trainer.evaluate()

    # Exibir os resultados da avaliação
    print("Resultados da Avaliação:", eval_results)

**Avaliação**

Para cada modelo é realizado a avaliação.

Este notebook foi mais de curiosidade para saber como o modelo se comporta a cada parte do dataset treinado. Deu para ver os seguintes comportamentos no modelo com todo o dataset:

- **Desempenho:** Apresentou uma perda de avaliação mais baixa.
- **Eficiência:** Nas amostras e passos processados por segundo.
- **Rapidez:** A avaliação foi mais rápida mesmo tendo sido treinado com um volume maior de dados.

In [7]:
for _, modelo in enumerate(modelos):
    avaliar_e_limpar_memoria(modelo)
    
print("Avaliação concluído.")

Avaliando o modelo: ./model/lora_model_parte_1
==((====))==  Unsloth 2024.8: Fast Llama patching. Transformers = 4.43.3.
   \\   /|    GPU: NVIDIA GeForce RTX 4070 Ti. Max memory: 11.994 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0+cu121. CUDA = 8.9. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.26.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Unsloth 2024.8 patched 22 layers with 22 QKV layers, 22 O layers and 22 MLP layers.


Map (num_proc=8):   0%|          | 0/179922 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


Resultados da Avaliação: {'eval_loss': 1.5356433391571045, 'eval_model_preparation_time': 0.0047, 'eval_runtime': 2473.3637, 'eval_samples_per_second': 72.744, 'eval_steps_per_second': 9.093}
Avaliando o modelo: ./model/lora_model_parte_2
==((====))==  Unsloth 2024.8: Fast Llama patching. Transformers = 4.43.3.
   \\   /|    GPU: NVIDIA GeForce RTX 4070 Ti. Max memory: 11.994 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0+cu121. CUDA = 8.9. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.26.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


...

max_steps is given, it will override any value given in num_train_epochs


Resultados da Avaliação: {'eval_loss': 1.4238680601119995, 'eval_model_preparation_time': 0.0051, 'eval_runtime': 2446.0835, 'eval_samples_per_second': 73.555, 'eval_steps_per_second': 9.195}
Avaliando o modelo: ./model/lora_model_final
==((====))==  Unsloth 2024.8: Fast Llama patching. Transformers = 4.43.3.
   \\   /|    GPU: NVIDIA GeForce RTX 4070 Ti. Max memory: 11.994 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0+cu121. CUDA = 8.9. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.26.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Map (num_proc=8):   0%|          | 0/179922 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


Resultados da Avaliação: {'eval_loss': 1.4180196523666382, 'eval_model_preparation_time': 0.005, 'eval_runtime': 2385.5828, 'eval_samples_per_second': 75.421, 'eval_steps_per_second': 9.428}
Avaliação concluído.
